In [316]:
import requests
from bs4 import BeautifulSoup
import json
from datetime import datetime
import re
def get_url():
    url = f"https://core.ac.uk/search?q=Shoubra+faculty+of+engineering&page=1"
    collected =[]
    i = 1
    classes = ["card-container-11P0y card-outlined-1ScI1 styles-container-2Sli1 styles-full-text-hosted-Zi-vo styles-container-recommended-2a9gS styles_search-results__2AZDM",'card-container-11P0y card-outlined-1ScI1 styles-container-2Sli1 styles-full-text-missing-itR-a styles_search-results__2AZDM',"card-container-11P0y card-outlined-1ScI1 styles-container-2Sli1 styles-full-text-hosted-Zi-vo styles_search-results__2AZDM","card-container-11P0y card-outlined-1ScI1 styles-container-2Sli1 styles-full-text-linked-1RAey styles_search-results__2AZDM"]
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
    old =0
    new = 1
    while old != new:
        new_url = url[:-1]+str(i)
        i+=1
        response = requests.get(new_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        old = len(collected)
        for cls in classes:
            a =  soup.find_all('div',class_=cls) 
            for link in a :
                collected.append(link.find('a')['href'])
        new = len(collected)
    return collected
    
def get_data(url):
    data = {}
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the <script> element with id="__NEXT_DATA__"
        script_element = soup.find('script', id='__NEXT_DATA__')

        if script_element:
            # Extract the JSON data from the <script> element
            script_content = script_element.string
            json_data = json.loads(script_content)

            # Extract the desired dictionaries directly
            props_dict = json_data.get('props', {})
            page_props_dict = props_dict.get('pageProps', {})
            data_dict = page_props_dict.get('data', {})

        else:
            print("Script element with id='__NEXT_DATA__' not found.")
            return data  # Return an empty dictionary

    else:
        print("Failed to fetch the URL. Status code:", response.status_code)
        return data  # Return an empty dictionary

    # Format authors
    authors = []
    for i in data_dict["outputs"][0]['authors']:
        authors.append(i['name'].replace(',', ""))

    data_dict['authors'] = authors

    # Check if 'publishedDate' key exists before using it
    datetime_string = data_dict.get('publishedDate', '')
    if datetime_string:
        datetime_object = datetime.strptime(datetime_string, '%Y-%m-%dT%H:%M:%S')
        formatted_datetime = datetime_object.strftime('%d %B %Y')
        data_dict['publishedDate'] = formatted_datetime

    cols = ['title', 'publishedDate', 'yearPublished', 'doi', 'abstract', 'fieldOfStudy']

    data['authors'] = authors
    for col in cols:
        data[col] = data_dict.get(col, '')  # Check if the key exists before using it

    return data
urls = get_url()
merged_list= []
for url in urls:
    data = get_data(url)
    merged_list.append(data)

df = pd.DataFrame(merged_list)

In [318]:
for url in urls:
    data = get_data(url)
    merged_list.append(data)

df = pd.DataFrame(merged_list)

Failed to fetch the URL. Status code: 504


In [319]:
df

,title,publishedDate,yearPublished,doi,abstract,fieldOfStudy,authors
0,A Business Analysis Perspective for Engineerin...,16 March 2020,2020.0,10.11114/jets.v8i5.4721,This work examines engineering education in Eg...,None,"[Elsafty Ashraf, Sayad Hanaa El, Shaaban Ibrahim]"
1,Educating engineering students in Egypt: a sta...,13 July 2020,2020.0,10.11114/jets.v8i8.4901,This is part of a series of three papers in a ...,None,"[Elsafty Ashraf Saeed, ElSayad Hanaa, Shaaban ..."
2,Effect of Coating of Earthed Enclosure and Mul...,01 August 2014,2014.0,10.11591/ijece.v4i4.5693,Metallic particle contamination is one of the ...,None,"[Abd Allah Mousa Awad Allah, Ward Sayed Abo El..."
3,A MILP model for an integrated project schedul...,01 January 2017,2017.0,10.1016/j.ifacol.2017.08.2221,"In this paper, we integrate two decision probl...",engineering,"[Karam Ahmed, Attia El-Awady, Duquenne Philippe]"
4,Effect of Functionally Graded Material of Disc...,01 December 2013,2013.0,10.11591/ijece.v3i6.4354,Solid insulators play a crucial role of electr...,None,"[Abd Allah Mousa Awad Allah, Ward Sayed Abo El..."
...,...,...,...,...,...,...,...
164,Multivariate discretization of continuous valu...,01 December 2006,2006.0,None,The area of Knowledge discovery and data minin...,None,[Ahmed Ehab Ahmed El Sayed 1978-]
165,Strength of eccentrically loaded slender colum...,01 October 2021,2021.0,10.1680/jstbu.19.00064,A non-linear finite-element analysis model was...,None,"[Shaaban Ibrahim, Abdel-Rahman Gamal T., Said ..."
166,Strength of eccentrically loaded slender colum...,05 September 2019,2019.0,10.1680/jstbu.19.00064,A non-linear finite-element analysis model was...,None,"[Shaaban Ibrahim G, Abdel-Rahman Gamal T, Said..."
167,Shear strength of reinforced mortar beams cont...,13 May 2021,2021.0,10.1007/s40999-021-00628-6,The current study aims to assess the shear beh...,None,"[Shaaban Ibrahim, Said Mohamed, Montaser Wael,..."


In [ ]:
urls = get_url()

In [ ]:

merged_list= []
for url in urls:
    data = get_data(url)
    merged_list.append(data)

df = pd.DataFrame(merged_list)

In [320]:
df.to_excel('dataset.xlsx', index=False)